# Treinamento FIWARE Lab @ RNP

## Desenvolvimento de aplicações com FIWARE com FIoT-Client

### Arquivo de configuração

In [1]:
def print_file(filename):
    with open(filename, 'r') as fin:
        print(fin.read())

In [2]:
print_file('config.ini')

[service]
fiware_service=UFRN
fiware_service_path=/dimap

[context_broker]
host=fiwarelab.rnp.br
port=8004
OAuth=no

[iota]
host=fiwarelab.rnp.br
north_port=8007
protocol_port=8006
OAuth=no
api_key=

[sth_comet]
host=fiwarelab.rnp.br
port=8666

[cygnus]
host=fiwarelab.rnp.br
port=5050
notification_host=fiwarelab.rnp.br

[mqtt_broker]
host=fiwarelab.rnp.br
port=8005

[perseo]
host=fiwarelab.rnp.br
port=9090

[local]
host_type=MacOSX
host_id=b4:b6:30



### Configuração dos clientes

In [3]:
from fiotclient.context import FiwareContextClient
from fiotclient.iot import FiwareIotClient
import json

client_context = FiwareContextClient.from_config_file('config.ini')
client_iot = FiwareIotClient.from_config_file('config.ini')

### Criação do serviço

In [4]:
response = client_iot.create_service('UFRN', '/imd')
api_key = response['api_key']
print('API Key: {}'.format(api_key))

API Key: 0188929677a911e9bf2c60f81db4b630


In [5]:
client_context.set_service('UFRN', '/imd')
client_iot.set_service('UFRN', '/imd')
client_iot.set_api_key(api_key)

### Criação de entidade

In [6]:
print_file('room_schema.json')

{
  "id": "[ENTITY_ID]",
  "type": "[ENTITY_TYPE]",
  "temperature": {
    "value": "",
    "type": "Float"
  },
  "pressure": {
    "value": "",
    "type": "Integer"
  }
}


In [7]:
client_context.create_entity_from_file('room_schema.json', 'Room', 'Room001')

{'status_code': 422,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '56', 'Connection': 'keep-alive', 'Fiware-Correlator': '01dc5bba-77a9-11e9-9fcb-0242ac11000b'},
 'response': {'error': 'Unprocessable', 'description': 'Already Exists'}}

### Listar entidades

In [8]:
client_context.get_entities()

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '1540', 'Connection': 'keep-alive', 'Fiware-Correlator': '01fa3586-77a9-11e9-aa36-0242ac11000b'},
 'response': [{'id': 'Room001',
   'type': 'Room',
   'pressure': {'type': 'Integer', 'value': '', 'metadata': {}},
   'temperature': {'type': 'Float', 'value': '', 'metadata': {}}},
  {'id': 'RoomSPLACE',
   'type': 'thing',
   'TimeInstant': {'type': 'ISO8601',
    'value': '2019-05-16T07:02:55.00Z',
    'metadata': {}},
   'change_state_ac_info': {'type': 'commandResult',
    'value': ' ',
    'metadata': {}},
   'change_state_ac_status': {'type': 'commandStatus',
    'value': 'PENDING',
    'metadata': {'TimeInstant': {'type': 'ISO8601',
      'value': '2019-05-16T07:02:55.601Z'}}},
   'change_temperature_ac_info': {'type': 'commandResult',
    'value': ' ',
    'metadata': {}},
   'change_temperature_ac_status': {'type': 'commandSta

### Filtrar entidades

Por tipo

In [9]:
client_context.get_entities_by_type('Room')

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '143', 'Connection': 'keep-alive', 'Fiware-Correlator': '021aa4f6-77a9-11e9-ae24-0242ac11000b'},
 'response': [{'id': 'Room001',
   'type': 'Room',
   'pressure': {'type': 'Integer', 'value': '', 'metadata': {}},
   'temperature': {'type': 'Float', 'value': '', 'metadata': {}}}]}

Por id

In [10]:
client_context.get_entity_by_id('Room001', 'Room')

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '141', 'Connection': 'keep-alive', 'Fiware-Correlator': '02293dae-77a9-11e9-b303-0242ac11000b'},
 'response': {'id': 'Room001',
  'type': 'Room',
  'pressure': {'type': 'Integer', 'value': '', 'metadata': {}},
  'temperature': {'type': 'Float', 'value': '', 'metadata': {}}}}

### Registro de dispositivo

In [11]:
print_file('splace_device_schema.json')

{
  "devices":[
    {
      "device_id":"[DEVICE_ID]",
      "entity_name":"[ENTITY_ID]",
      "entity_type":"thing",
      "timezone":"America/Fortaleza",
      "protocol":"IoTA-UL",
      "transport":"MQTT",
      "attributes":[
        {
          "object_id":"t",
          "name":"temperature",
          "type":"int"
        },
        {
          "object_id":"h",
          "name":"humidity",
          "type":"int"
        },
        {
          "object_id":"p",
          "name":"presence",
          "type":"bool"
        },
        {
          "object_id":"np",
          "name":"num_people",
          "type":"int"
        },
        {
          "object_id":"sac",
          "name":"state_ac",
          "type":"bool"
        },
        {
          "object_id":"tac",
          "name":"temperature_ac",
          "type":"int"
        }
      ],
      "commands":[
        {
          "name":"change_state_ac",
          "type":"command",
          "value":"[DEVICE_ID]@change_state_ac|%s

In [12]:
client_iot.register_device_from_file('splace_device_schema.json', 'SPLACE001', 'RoomSPLACE')

{'status_code': 409,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '110', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Fiware-Correlator': 'bb1493a3-4b50-4325-81d9-d4a2e84df1d4', 'ETag': 'W/"6e-WddhtiQc8ikjEqrC44yazr5os70"'},
 'response': {'name': 'DUPLICATE_DEVICE_ID',
  'message': 'A device with the same pair (Service, DeviceId) was found:SPLACE001'}}

### Listar dispositivos

In [13]:
client_iot.list_devices()

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '848', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Fiware-Correlator': '0e1823dc-4094-4a33-bdc2-10499ef22a10', 'ETag': 'W/"350-sHDURmCDIzP5w0mQRdpS93AfE84"'},
 'response': {'count': 1,
  'devices': [{'device_id': 'SPLACE001',
    'service': 'UFRN',
    'service_path': '/imd',
    'entity_name': 'RoomSPLACE',
    'entity_type': 'thing',
    'transport': 'MQTT',
    'attributes': [{'object_id': 't', 'name': 'temperature', 'type': 'int'},
     {'object_id': 'h', 'name': 'humidity', 'type': 'int'},
     {'object_id': 'p', 'name': 'presence', 'type': 'bool'},
     {'object_id': 'np', 'name': 'num_people', 'type': 'int'},
     {'object_id': 'sac', 'name': 'state_ac', 'type': 'bool'},
     {'object_id': 'tac', 'name': 'temperature_ac', 'type': 'int'}],
    'lazy': [],
    'commands': [{'object_id': 'change_state_a

### Envio de dados

Usando UL-HTTP

In [14]:
client_iot.send_observation('SPLACE001', {'t': 36, 'h': 100, 'p': 1}, protocol='HTTP')

{'result': 'OK'}

### Execução de comandos

In [15]:
client_iot.send_command('RoomSPLACE', 'SPLACE001', 'change_state_ac', {'state': 'ON'})

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Content-Length': '210', 'Connection': 'keep-alive', 'X-Powered-By': 'Express', 'Fiware-Correlator': 'dc85fea7-42e7-4599-8190-8e46937d78ca', 'ETag': 'W/"d2-Qf7s59QD5yddhBZVhXtE8lHjm9I"'},
 'response': {'contextResponses': [{'contextElement': {'attributes': [{'name': 'change_state_ac',
       'type': 'command',
       'value': ''}],
     'id': 'RoomSPLACE',
     'isPattern': False,
     'type': 'thing'},
    'statusCode': {'code': 200, 'reasonPhrase': 'OK'}}]}}

### Armazenamento de dados históricos de leituras

In [16]:
client_context.subscribe_cygnus('RoomSPLACE', ['temperature', 'humidity', 'presence'])

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '104', 'Connection': 'keep-alive', 'Fiware-Correlator': '0275c5ca-77a9-11e9-8df6-0242ac11000b'},
 'response': {'subscribeResponse': {'subscriptionId': '5cdd0bc223832999dee105b8',
   'duration': 'P1Y',
   'throttling': 'PT1S'}}}

In [17]:
client_context.list_subscriptions()

{'status_code': 200,
 'headers': {'Server': 'nginx/1.12.2', 'Date': 'Thu, 16 May 2019 07:05:38 GMT', 'Content-Type': 'application/json', 'Content-Length': '932', 'Connection': 'keep-alive', 'Fiware-Correlator': '0281fcb4-77a9-11e9-8e3b-0242ac11000b'},
 'response': [{'id': '5cdd0b6823832999dee105b7',
   'expires': '2020-05-15T07:04:08.00Z',
   'status': 'failed',
   'subject': {'entities': [{'id': 'RoomSPLACE', 'type': 'thing'}],
    'condition': {'attrs': ['temperature', 'humidity', 'presence']}},
   'notification': {'timesSent': 2,
    'lastNotification': '2019-05-16T07:05:38.00Z',
    'attrs': ['temperature', 'humidity', 'presence'],
    'attrsFormat': 'legacy',
    'http': {'url': 'http://fiwarelab.rnp.br:5050/notify'},
    'lastFailure': '2019-05-16T07:05:38.00Z',
    'lastFailureReason': "Couldn't connect to server"},
   'throttling': 1},
  {'id': '5cdd0bc223832999dee105b8',
   'expires': '2020-05-15T07:05:38.00Z',
   'status': 'active',
   'subject': {'entities': [{'id': 'RoomSPL